# Exploring GTFS data and creating a public transportation network


The General Transit Feed Specification (GTFS), also known as GTFS static or static transit to differentiate it from the GTFS realtime extension, defines a common format for public transportation schedules and associated geographic information. GTFS "feeds" let public transit agencies publish their transit data and developers write applications that consume that data in an interoperable way.

https://developers.google.com/transit/gtfs/

In [1]:
import networkx as nx
import pandas as pd
import geopandas as gpd
from shapely.geometry import *
import os
import numpy as np

### Look at the available files

these are all the files that make Israel's entire public transportation network lines and their schedules the data can be found here:

https://www.gov.il/he/Departments/General/gtfs_general_transit_feed_specifications

In [2]:
files = os.listdir('../data/israel-public-transportation/')
files

['agency.txt',
 'calendar.txt',
 'fare_attributes.txt',
 'fare_rules.txt',
 'routes.txt',
 'shapes.txt',
 'stops.txt',
 'stop_times.txt',
 'translations.txt',
 'trips.txt']

### Data Description
<ul>
<li> agency.txt	Required	Defines one or more transit agencies that have services represented in the dataset.</li>
<li>stops.txt	Required	Defines stops where vehicles pick up or drop off riders. Also defines stations and station entrances.</li>
<li>routes.txt	Required	Defines transit routes. A route is a group of trips that are displayed to riders as a single service.</li>
<li>trips.txt	Required	Defines trips for each route. A trip is a sequence of two or more stops that occur during a specific time period.</li>
<li>stop_times.txt	Required	Provides the times when a vehicle arrives at and departs from individual stops for each trip.</li>
<li>calendar.txt	Conditionally required	Defines service dates when service is available for particular routes. Uses a weekly schedule. This file specifies start and end dates of service, as well as the days of the week when service is available. This file is required unless all of the dates of service are defined in calendar_dates.txt.</li>
<li>calendar_dates.txt	Conditionally required	Defines exceptions for the services defined in the calendar.txt file. If calendar.txt is omitted, then calendar_dates.txt is required and must contain all dates of service.</li>
<li>fare_attributes.txt	Optional	Defines fare information for a transit agency's routes.</li>
<li>fare_rules.txt	Optional	Defines rules to apply fare information for itineraries.</li>
<li>shapes.txt	Optional	Defines rules to map vehicle travel paths, sometimes referred to as route alignments.</li>
<li>frequencies.txt	Optional	Describes the headway, which is the time between trips, for routes that have a variable frequency of service. Can also provide a compressed representation of fixed-schedule services.</li>
<li>transfers.txt	Optional	Defines rules for how connections are made at transfer points between routes.</li>
<li>pathways.txt	Optional	Defines rules to describe the walking connections between two stops within a station, such as between an entrance and a platform. These pathways are represented as the edges of a walking graph.</li>
<li>levels.txt	Optional	Defines levels within stations.
feed_info.txt	Optional	Provides additional information about the feed itself, such as publisher, version, and expiration information.</li>
    </ul>

#### Lets look at the table of all the pt stops in Israel

In [3]:
stops = pd.read_csv('../data/israel-public-transportation/stops.txt')
stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,zone_id
0,1,38831,בי''ס בר לב/בן יהודה,רחוב:בן יהודה 76 עיר: כפר סבא רציף: קומה:,32.183939,34.917812,0,NaN,6900
1,2,38832,הרצל/צומת בילו,רחוב:הרצל עיר: קרית עקרון רציף: קומה:,31.870034,34.819541,0,NaN,469
2,3,38833,הנחשול/הדייגים,רחוב:הנחשול 30 עיר: ראשון לציון רציף: קומה:,31.984553,34.782828,0,NaN,8300
3,4,38834,פריד/ששת הימים,רחוב:משה פריד 9 עיר: רחובות רציף: קומה:,31.888550,34.790904,0,NaN,8400
4,6,38836,ת. מרכזית לוד/הורדה,רחוב: עיר: לוד רציף: קומה:,31.956576,34.898125,0,NaN,7000


In [4]:
stops = gpd.GeoDataFrame(stops, geometry = [ Point([x['stop_lon'], x['stop_lat']]) for _, x in stops.iterrows()], 
                        crs = {'init':'epsg:4326'})

In [5]:
stops[stops['zone_id']==3000].plot(figsize = (10,10))

#### The shapes are the geometries that make up the routes

In [6]:
shapes = pd.read_csv('../data/israel-public-transportation/shapes.txt')
shapes.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,44779,31.887695,35.016271,1
1,44779,31.887745,35.016253,2
2,44779,31.888256,35.016238,3
3,44779,31.888913,35.016280,4
4,44779,31.888917,35.016892,5


#### Optional: 
    create a geodataframe from the shapes table - we will not need this for this particular analysis

In [7]:

# lines = []
# ids = []
# for i in shapes['shape_id'].unique():
    
#     ## create a dataframe for each shape
#     df2 = shapes[shapes['shape_id']==i].copy()
    
#     ### sort by sequence value (just in case)
#     df2 = df2.sort_values('shape_pt_sequence')
    
#     ### create a list of lon, lat tuples for each point along the shape 
    
#     l = [(x['shape_pt_lon'],x['shape_pt_lat'] ) for _, x in df2.iterrows()]

#     #### create a line from the points
#     l = LineString(l)
#     ### append to the lines list the shape_id, 
#     lines.append(l)
#     ids.append(i)

# ###put these all together in one data frame 
# new_shapes =gpd.GeoDataFrame({
    
#     'shape_id':ids,
#     'geometry':lines
    
# }) 

In [ ]:
new_shapes.plot(figsize = (15,15))

#### Lets look at all the routes (route_short_id is what we know as the line number)

In [13]:
routes = pd.read_csv('../data/israel-public-transportation/routes.txt')
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color
0,1,25,1,ת. רכבת יבנה מערב-יבנה<->ת. רכבת יבנה מזרח-יבנ...,67001-1-#,3,NaN
1,2,25,1,ת. רכבת יבנה מזרח-יבנה<->ת. רכבת יבנה מערב-יבנ...,67001-2-#,3,NaN
2,3,25,2,ת. רכבת יבנה מערב-יבנה<->ת. רכבת יבנה מזרח-יבנ...,56002-1-#,3,NaN
3,5,25,2,ת. רכבת יבנה מזרח-יבנה<->ת. רכבת יבנה מערב-יבנ...,56002-2-#,3,NaN
4,7,25,3,ת. רכבת יבנה מערב-יבנה<->ת. רכבת יבנה מזרח-יבנ...,49003-1-#,3,NaN


In [14]:
routes[routes['route_short_name']=='826']

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color
1863,7072,3,826,ת.מרכזית ת''א ק.7/רציפים-תל אביב יפו<->חניון נ...,10826-1-#,3,NaN
1864,7079,3,826,הגליל/הגולן-נופית<->ת. מרכזית ת''א ק. 7/הורדה-...,10826-2-ז,3,NaN
1865,7081,3,826,שדרות הבנים/נוף העמק-מגדל העמק<->ת. מרכזית ת''...,10826-2-ט,3,NaN
1866,7083,3,826,דליה/כליל החורש-נצרת עילית<->ת. מרכזית ת''א ק....,10826-2-כ,3,NaN
4797,15123,3,826,ת.מרכזית ת''א ק.7/רציפים-תל אביב יפו<->דליה-נצ...,10826-1-9,3,NaN
6704,19731,3,826,חניון אגד/העבודה-נצרת עילית<->ת. מרכזית ת''א ק...,10826-2-#,3,NaN


#### trips are all the occurrences of every route

In [15]:
trips = pd.read_csv('../data/israel-public-transportation/trips.txt')
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id
0,1,1,30900053_180219,רכבת מזרח/שוק,0,97105.0
1,1,1,30900054_180219,רכבת מזרח/שוק,0,97105.0
2,1,1,30900055_180219,רכבת מזרח/שוק,0,97105.0
3,1,1,30900056_180219,רכבת מזרח/שוק,0,97105.0
4,1,1,30900057_180219,רכבת מזרח/שוק,0,97105.0


lets use the calendar table in order to filter in:
    1. only trips that are active on Mondays - we will use this a general representation of a weekday schedule
    2. where trips schedules change over the course of the calendar time period - we will use only the first

In [16]:
calendar = pd.read_csv('../data/israel-public-transportation/calendar.txt')
calendar.head()

,service_id,sunday,monday,tuesday,wednesday,thursday,friday,saturday,start_date,end_date
0,1,1,1,1,1,1,0,0,20190218,20190319
1,2,0,0,0,0,0,1,0,20190218,20190319
2,3,0,0,0,0,0,0,1,20190218,20190319
3,4,0,0,0,1,0,0,0,20190320,20190320
4,5,0,0,0,0,1,0,0,20190321,20190321


get all the services_ids that occur on a Monday

In [17]:
monday_services = set(calendar[calendar['monday'] == 1]['service_id'])

filter the trips table

In [18]:
len(trips)

251691

In [19]:
trips = trips[trips['service_id'].apply(lambda x: x in  monday_services)]

In [20]:
len(trips)

129147

split the trip_id string up to the start calendar date

In [21]:
trips['ids'] = trips['trip_id'].apply(lambda x: x[:x.index('_')])

lets look at the trips grouped by the new ids. we can see that some ids have more than 1 occurrence:


In [22]:
trips.groupby('ids').size().sort_values(ascending = False).head()

ids
33563691    4
33563782    4
33563801    4
33563800    4
33563799    4
dtype: int64

lets look at the first duplicated id in the trip table

In [23]:
trips[trips['ids']== '33563691' ]

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id,ids
139645,11685,13047,33563691_180219,קרית מוצקין_מרכזית הקריות,0,96286.0,33563691
140218,11685,13051,33563691_260219,קרית מוצקין_מרכזית הקריות,0,96286.0,33563691
140431,11685,13052,33563691_280219,קרית מוצקין_מרכזית הקריות,0,103554.0,33563691
140809,11685,13055,33563691_030319,קרית מוצקין_מרכזית הקריות,0,103554.0,33563691


lets look at these rervices in the calendar table
we can see that the trip schedule changes between 18/02/2019 to 19/04/2019

In [24]:
set_ = set(trips[trips['ids']== '33563691' ]['service_id'])

In [25]:
set_

{13047, 13051, 13052, 13055}

In [26]:
calendar[calendar['service_id'].apply(lambda x: x in set_)]

,service_id,sunday,monday,tuesday,wednesday,thursday,friday,saturday,start_date,end_date
13046,13047,1,1,1,1,1,0,0,20190218,20190224
13050,13051,1,1,1,1,1,0,0,20190226,20190227
13051,13052,1,1,1,1,1,0,0,20190228,20190301
13054,13055,1,1,1,1,1,0,0,20190303,20190419


We can filter the first occurrence only by using groupby and head(1)

In [27]:
len(trips)

129147

In [28]:
trips = trips.groupby('ids').head(1)

In [29]:
len(trips)

102127

get a set of the filtered trip_ids

In [30]:
monday_first_date_trips = set(trips[trips['service_id'].apply(lambda x: x in  monday_services)]['trip_id'])

#### The stop_times table is the actual schedule

we will use this to build our network - we will discard the geometries but keep the distances from 
this table

In [31]:
stop_times = pd.read_csv('../data/israel-public-transportation/stop_times.txt')
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled
0,10021427_180219,00:00:00,00:00:00,36133,1,0,1,0
1,10021427_180219,00:01:35,00:01:35,2356,2,0,1,499
2,10021427_180219,00:03:27,00:03:27,3869,3,0,1,1838
3,10021427_180219,00:05:22,00:05:22,2357,4,0,1,2693
4,10021427_180219,00:06:23,00:06:23,3803,5,0,1,3379


filter for monday first date trips

In [32]:
stop_times = stop_times[stop_times['trip_id'].apply(lambda x: x in monday_first_date_trips)].copy()
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled
415,10096398_180219,21:00:00,21:00:00,34657,1,0,1,0
416,10096398_180219,21:00:40,21:00:40,35317,2,0,0,276
417,10096398_180219,21:01:56,21:01:56,34436,3,0,0,857
418,10096398_180219,21:04:18,21:04:18,34444,4,0,0,1955
419,10096398_180219,21:14:43,21:14:43,35506,5,0,0,14838


In order create an edge table (or edge list) from the stop_times table 
append the next row stop to every row (the stops are ordered in trip sequences)
trip_id - is the trip_id from the trips table
stop_sequence - is the order of the stops on the trip

note:
1. we start from the second element and add another element at the end -  offsetting each row
2. we will later remove the last stop in every trip

In [33]:
# reset the index
stop_times = stop_times.reset_index(drop = True)

In [34]:
stop_times['to'] = stop_times.loc[1:, 'stop_id'].tolist()+[0]

here we are adding the distance to the next stop by along the trip again  offsetting by one row

we also deduct the distance already traveled along the trip to get only the distance between two stops

In [35]:
stop_times['distance'] = stop_times.loc[1:, 'shape_dist_traveled'].tolist()+[np.nan]
stop_times['distance'] = stop_times['distance']-stop_times['shape_dist_traveled']

we calculate a stop sequence offset and if it is different than 1 it means we are at the last stop in the trip

we would want to remove these rows

In [36]:
stop_times['offset'] = stop_times.loc[1:, 'stop_sequence'].tolist()+[np.nan]
stop_times['offset'] = stop_times['offset']- stop_times['stop_sequence']

In [37]:
stop_times = stop_times[stop_times['offset']==1]

In [38]:
len(stop_times)

3500793

### Create the network

we want our network to express every time a train or a bus stops at a stop (there can be many duplicate edges) - so we will work with 
a directed multi graph

In [39]:
G=nx.MultiDiGraph()

we add the edges and the distances between them as the weight

In [40]:
G.add_weighted_edges_from(zip(stop_times['stop_id'], stop_times['to'], stop_times['distance']))


In [41]:
nx.is_directed(G)

True

In [42]:
list(G.edges(data = True))[0]

(34657, 35317, {'weight': 276.0})

We want to add node names and zones to our network. We can create dictionaries of stop_ids and any other attribute and add them to the network as node attributes - we will use the stop table for this, because it contains unique nodes attributes


In [43]:
nodes_dict = dict(zip(stops['stop_id'], stops['stop_name']))
zones_dict = dict(zip(stops['stop_id'], stops['zone_id']))

In [44]:

nx.set_node_attributes(G,nodes_dict, "name")
nx.set_node_attributes(G,zones_dict, "zone")

In [45]:
G.nodes(data = True)[1]

{'name': "בי''ס בר לב/בן יהודה", 'zone': 6900}

### create a subgraph of a zone

Now that are nodes have zone_id s we can subset only the part of the network that
is in the location we are looking at (zone_id are the locality id as the appear in the lamas census) - 

The advantage of subsetting the graph instead of filtering the original dataframe from which the graph was prepared,
is that here we get only the parts of trips that are within the specified zone. Otherwize we can include trips going from the zone to another - and in this specific case - this is not what we want

In [46]:
SG = G.subgraph([i for i in G.nodes if G.nodes[i]['zone'] == 3000])

### Calculation centrality measures for our network

In [47]:
dc = nx.degree_centrality(SG)
print('degree centrality finished')
d = dict(SG.degree())
print('degree finished')
d_out = dict(SG.out_degree())
print('out degree finished')
d_in = dict(SG.in_degree())
print('in degree finished')
# cc = nx.closeness_centrality(SG, distance = 'distance')
# print('closeness centrality finished')
bc = nx.betweenness_centrality(SG, k= 200)
print('betweenness centrality finished')


degree centrality finished
degree finished
out degree finished
in degree finished
betweenness centrality finished


page rank and hits will not work on a multigraph
we will convert do directed graph and add the out degree as an attribute to be used as weight

In [48]:
SG1 = nx.DiGraph(SG)
nx.set_node_attributes(SG1, dict(d_out), 'out_degree')
pr = nx.pagerank(SG1,weight= 'out_degree' )
hits = nx.hits(SG1, normalized = True)



lets look at the top 5 in degrees in the network

In [49]:
degrees = sorted(SG.in_degree(), key=lambda tup: tup[1], reverse = True)

top_5 = [(SG.nodes[i[0]]['name'],i[0], i[1]) for i in degrees[0:5]]

top_5

[('ירמיהו/מנחת יצחק', 10721, 1964),
 ('ירמיהו/אלקנה', 10785, 1906),
 ('ירמיהו/אלקנה', 10720, 1747),
 ('ירמיהו/מנחת יצחק', 10784, 1681),
 ('בר אילן/רבנו גרשום', 10786, 1676)]

### Add the centrality measures to the stops table

subset to the region of interest

In [50]:
stops_jer = stops[stops['zone_id']== 3000].copy()

In [51]:
len(stops_jer)

2030

add the data to the dataframe

In [52]:

stops_jer['in_degree'] = [d_in[x]  if x in d_in.keys() else 0  for x in stops_jer['stop_id']]
stops_jer['out_degree'] = [d_out[x]  if x in d_out.keys() else 0  for x in stops_jer['stop_id']]
stops_jer['dc'] = [dc[x]  if x in dc.keys() else 0  for x in stops_jer['stop_id']]
# stops_jer['cc'] = [cc[x]  if x in cc.keys() else 0  for x in stops_jer['stop_id']]
stops_jer['bc'] = [bc[x]  if x in bc.keys() else 0  for x in stops_jer['stop_id']]
stops_jer['pr'] = [pr[x]  if x in pr.keys() else 0  for x in stops_jer['stop_id']]
stops_jer['hb'] = [hits[0][x]  if x in hits[0].keys() else 0  for x in stops_jer['stop_id']]
stops_jer['au'] = [hits[1][x]  if x in hits[1].keys() else 0  for x in stops_jer['stop_id']]

In [53]:
stops_jer.sort_values(by = 'pr', ascending = False).head(20)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,zone_id,geometry,in_degree,out_degree,dc,bc,pr,hb,au
8267,10716,4009,צומת רמות/גולדה,רחוב:שדרות גולדה מאיר עיר: ירושלים רציף: ק...,31.808251,35.203986,0,NaN,3000,POINT (35.203986 31.808251),1187,1219,1.189911,0.149633,0.002853,5.643326e-02,2.073510e-02
6952,8916,491,מוזיאון רוקפלר/סולטאן סולימאן,רחוב:סולטאן סולימאן 23 עיר: ירושלים רציף: ק...,31.783533,35.234357,0,NaN,3000,POINT (35.234357 31.783533),626,626,0.619189,0.034476,0.002487,1.701995e-95,5.011780e-92
8861,11394,5769,תחנה מרכזית סולטן סולימאן/רחוב סולטן סולימאן,רחוב: עיר: ירושלים רציף: קומה:,31.782927,35.231325,0,NaN,3000,POINT (35.231325 31.782927),723,93,0.403561,0.033599,0.002457,1.321371e-97,1.112012e-93
8327,10790,5130,צומת רמות/גולדה,רחוב:שדרות גולדה מאיר עיר: ירושלים רציף: ק...,31.809648,35.204063,0,NaN,3000,POINT (35.204063 31.809648),942,963,0.942136,0.033521,0.002292,3.338115e-04,9.496752e-02
7972,10264,3035,צומת רמות/ידין,רחוב:יציאה לרחוב יגאל ידין עיר: ירושלים רציף...,31.806282,35.204894,0,NaN,3000,POINT (35.204894 31.806282),278,248,0.260138,0.092247,0.001839,6.368142e-02,6.530821e-02
7457,9589,1625,חטיבת הראל/שמואל הנביא,רחוב:חטיבת הראל עיר: ירושלים רציף: קומה:,31.795977,35.220767,0,NaN,3000,POINT (35.220767 31.795977),991,991,0.980218,0.050510,0.001594,2.473189e-09,1.367351e-03
7384,9503,1435,שד' צ'רצ'יל/קציר,רחוב:שדרות צ'רצ'יל עיר: ירושלים רציף: קומה:,31.797858,35.238286,0,NaN,3000,POINT (35.238286 31.797858),499,499,0.493571,0.003408,0.001569,1.759422e-251,8.912231e-18
8382,10846,5200,בנייני האומה,רחוב:שדרות שז''ר עיר: ירושלים רציף: קומה:,31.787932,35.203427,0,NaN,3000,POINT (35.203427 31.787932),1420,806,1.100890,0.026533,0.001563,2.897107e-04,1.051992e-04
7831,10071,2729,טיילת ארמון הנציב/ינובסקי,רחוב:דניאל ינובסקי עיר: ירושלים רציף: קומה:,31.755138,35.227898,0,NaN,3000,POINT (35.227898 31.755138),244,245,0.241840,0.052849,0.001473,1.392538e-132,7.097300e-119
8085,10452,3296,דרך חברון/גבעת המטוס,רחוב: עיר: ירושלים רציף: קומה:,31.740490,35.213263,0,NaN,3000,POINT (35.213263 31.74049),643,648,0.638477,0.028649,0.001470,9.661884e-08,3.283790e-11


save to file:

In [54]:

stops_jer.to_file('../data/stops_jer.geojson', driver = 'GeoJSON')